#UE03 - SPARQL Query

Please complete the 10 tasks in the `2. SPARQL` sheet of `SemAI.jar` first, and then transfer the task descriptions and your solutiosn in executable form to this notebook.

## Preparation

Reuse imports and functions from https://github.com/jku-win-dke/SemAI/blob/main/V02_SPARQL.ipynb and load the solar system graph. Query the solar system graph to check that everything works fine. 

In [ ]:
# Install required packages
!pip install -q rdflib

# Imports
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import FOAF , XSD , RDFS, NamespaceManager 

# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def sparql_construct(graph, query):
  result_graph = Graph(namespace_manager = g.namespace_manager)  # create a Graph object that reuses the namespace prefixes of the original graph
  result_graph += graph.query(query)                             # execute the construct query against the original graph and add the resulting graph to the new one
  return result_graph

def sparql_ask(graph, query):
  return bool(graph.query(query))      # an ASK query has a boolean result, which should be returned as such

# Load solar system graph
g = Graph()
g.parse("https://raw.githubusercontent.com/jku-win-dke/SemAI/main/data/solarsystem.ttl",format="turtle")

# Query solar system graph (to check that everything works fine)
df = sparql_select(g,"""
  SELECT ?planet ?apoapsis ?apoapsis_uom
  WHERE { 
    ?planet rdf:type dbo:Planet . 
    OPTIONAL { ?planet v:apoapsis [rdf:value ?apoapsis ; v:uom ?apoapsis_uom ].  }
  }
""")
df

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 3.5 MB/s eta 0:00:00


,planet,apoapsis,apoapsis_uom
0,:Mercury,0.467,unit:AU
1,:Venus,0.728,unit:AU
2,:Earth,1.017,unit:AU
3,:Earth,149597871,unit:KM
4,:Mars,1.666,unit:AU
5,:Jupiter,5.4588,unit:AU
6,:Saturn,9.0412,unit:AU
7,:Uranus,20.11,unit:AU
8,:Neptune,30.33,unit:AU



## Task 1 (1 pt)

Geben Sie alle Zwergplaneten (Instanzen der Klasse dbo:DwarfPlanet) aus und falls vorhanden deren genaue Entsprechung (skos:exactMatch). Ordnen Sie das Ergebnis aufsteigend nach den URIs der Zwergplaneten. *

In [ ]:
query = '''
SELECT ?d ?match
WHERE { 
 ?d a dbo:DwarfPlanet .
 OPTIONAL {
  ?d skos:exactMatch ?match .
 }
}
ORDER BY ?d
'''

sparql_select(g, query)

,d,match
0,:Ceres,dbr:1_Ceres
1,:Eris,None
2,:Haumea,None
3,:Makemake,None
4,:Pluto,dbr:Pluto


## Task 2 (1 pt)

Ermitteln Sie alle Sterne, ihr exactMatch, und ihre Masse (geben Sie den Wert und die Maßeinheit aus). Ordnen Sie das Ergebnis nach der URI der Sterne 

In [ ]:
query = '''
SELECT ?star ?match ?massVal ?massUoM
WHERE { 
 ?star a dbo:Star . 
 ?star skos:exactMatch ?match .
 ?star v:mass ?massBlankNode .
 ?massBlankNode rdf:value ?massVal .
 ?massBlankNode v:uom ?massUoM .
}
ORDER BY ?star
'''

sparql_select(g, query)

,star,match,massVal,massUoM
0,:AlphaCentauriA,wd:Q2090157,1.1,v:SolarMass
1,:AlphaCentauriB,wd:Q1052548,0.9,v:SolarMass
2,:ProximaCentauri,wd:Q14266,0.1221,v:SolarMass
3,:Sun,dbr:Sun,1.9884e+30,unit:KG


## Task 3 (1 pt)

Die Planeten unseres Sonnensystems und ihre jeweilige Anzahl an Monden. Unterscheiden Sie die im RDF-Graph beschriebenen Monde und die im RDF-Graph erfasste Anzahl von Monden. Sortieren Sie nach den Planeten. 

In [ ]:
query = '''
SELECT ?planet ?assertedNo ?noOfDescribedMoons
WHERE { 
 ?planet a dbo:Planet .
 ?planet v:orbits :Sun .  
 ?planet v:nrOfMoons ?assertedNo .
 OPTIONAL {
  SELECT ?planet (count(?m) AS ?noOfDescribedMoons)
  WHERE { 
   ?planet a dbo:Planet .
   OPTIONAL {
    ?m v:orbits ?planet .
   }
  }
  GROUP BY ?planet
 }
}
ORDER BY ?planet
'''

sparql_select(g, query)

,planet,assertedNo,noOfDescribedMoons
0,:Earth,None,1
1,:Jupiter,None,4
2,:Mars,None,2
3,:Mercury,None,0
4,:Neptune,None,0
5,:Saturn,None,2
6,:Uranus,None,0
7,:Venus,None,0


## Task 4 (1 pt)

Ermitteln sie für die Planeten in unserem Sonnensystem die durchschnittliche Anzahl an Monden (die auch im RDF-Graph beschrieben sind) pro Planet. 

In [ ]:
query = '''
SELECT ((?moonCount / ?planetCount) AS ?avgNoOfDescribedMoons)
WHERE { 
 {
  SELECT (count(?planet) AS ?planetCount)
  WHERE {
   ?planet a dbo:Planet .
   ?planet v:orbits :Sun .
  }
 }
  
 {
  SELECT (count(?moon) AS ?moonCount)
  WHERE {
   ?planet a dbo:Planet .
   ?moon v:orbits ?planet .
  }
 }
}
'''

sparql_select(g, query)

,avgNoOfDescribedMoons
0,1.125


## Task 5 (1 pt)

Geben Sie die im RDF-Graph verwendeten Klassen und ihre Anzahl an Instanzen aus. Geben Sie nur Klassen mit mindestens 2 Instanzen aus. Ordnen Sie die Ausgabe nach der URI der Klassen. 

In [ ]:
query = '''
SELECT ?class (count(?class) AS ?noOfInstances)
WHERE { 
 ?r rdf:type ?class .
}
GROUP BY ?class
HAVING (?noOfInstances > 2)
ORDER BY ?class
'''

sparql_select(g, query)

,class,noOfInstances


## Task 6 (1 pt)

Geben Sie die im RDF-Graph verwendeten Properties und ihre Häufigkeit je Klasse aus. Gezählt werden sollen die Instanzen der Klasse, die die Property ausprägen. Geben Sie nur Properties zu Klassen aus, die von mindestens 7 Instanzen dieser Klasse ausgeprägt werden. Ordnen Sie die Ausgabe zuerst nach der Klasse und dann nach der Property. 

In [ ]:
query = '''
SELECT ?class ?prop (count(?prop) AS ?noOfInstances)
WHERE {
 {
  SELECT DISTINCT ?s ?class ?prop
  WHERE {
   ?s ?prop ?o .
   ?s rdf:type ?class .
  }
 }
}
GROUP BY ?class ?prop
HAVING (?noOfInstances > 6)
ORDER BY ?class ?prop
'''

sparql_select(g, query)

,class,prop,noOfInstances


## Task 7 (1 pt)

Ermitteln Sie alle Sterne, sowie alle Monde. Zu jedem Mond ermitteln Sie auch den Planet, den dieser Mond umrundet. Ordnen Sie die Ausgabe nach der URI der Sterne und Monde. 

In [ ]:
query = '''
SELECT ?x ?y
WHERE { 
 {
  SELECT ?x
  WHERE {
   ?x a dbo:Star .
  }
 }
 UNION
 {
  SELECT ?x
  WHERE {
   ?x a dbo:Satellite .
  }
 }

 OPTIONAL {
  ?x v:orbits ?y
 }
}
ORDER BY ?x ?y
'''

sparql_select(g, query)

,x,y
0,:AlphaCentauriA,None
1,:AlphaCentauriB,None
2,:Callisto,:Jupiter
3,:Deimos,:Mars
4,:Enceladus,:Saturn
5,:Europa,:Jupiter
6,:Ganymede,:Jupiter
7,:Io,:Jupiter
8,:Mimas,:Saturn
9,:Moon,:Earth


## Task 8 (1 pt)

Erzeugen sie mittels einer Construct-Query den angezeigten RDF-GRAPH (Expected Result). Reihenfolge ist unerheblich. 

In [ ]:
query = '''
CONSTRUCT {
 ?x a v:Himmelskoerper .
 ?y v:wirdUmrundetVon ?z .
}
WHERE {
 {
  SELECT ?x
  WHERE {
   ?x rdf:type dbo:Planet .
  }
 }
 UNION
 {
  SELECT ?x
  WHERE {
   ?x rdf:type dbo:Satellite .
  }
 }

 OPTIONAL {
  ?z v:orbits ?y .
 }
}
'''

sparql_construct(g, query)

<Graph identifier=N37ec59e11c9a4341b262dcdc69fe125f (<class 'rdflib.graph.Graph'>)>

## Task 9 (1 pt)

Gibt es einen Stern im RDF-Graph, der massereicher als die Sonne ist. 

In [ ]:
query = '''
ASK {
 SELECT ?x
 WHERE {
  ?x a dbo:Star .
  ?x v:mass ?compBlankNode .
  ?compBlankNode rdf:value ?compMassVal .
  ?compBlankNode v:uom v:SolarMass .
  FILTER(?compMassVal > 1)
 }
}
'''

sparql_ask(g, query)

True

## Task 10 (1 pt)

Geben Sie eine Beschreibung der Resourcen aus, die einen Durchmesser zwischen 20.000 km und 30.000 km haben. 

In [ ]:
query = '''
DESCRIBE ?x
WHERE {
 ?x v:radius ?radiusBlankNode .
 ?radiusBlankNode rdf:value ?radius .
 FILTER(?radius > 20000 && ?radius < 30000)
}
'''

print(sparql_construct(g, query))

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].
